In [ ]:
from train import train
from inference import inference
from save_mask import save_mask
import csv, os
from utils.module import write_to_csv

In [ ]:
!nvidia-smi

In [ ]:
# Uncertainty-based Self-Training
project = "240208"
folnames = [project+"_iter1", project+"_iter2", project+"_iter3", project+"_iter4", project+"_iter5"]
os.makedirs('results/'+ project +'_results', exist_ok=True)
csv_filename = 'results/'+project+'_results/results.csv'
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Iteration', 'F1-Score', 'Accuracy', 'Specificity', 'Recall', 'Precision'])
    
# params setting
JsonDataSplit = False
target_dataset = 'chun'
useStableDiffusion = False
net = 'deeplab'
cons_reg = False
alpha = 100

for i in range(len(folnames)):
    if i==0:
        train(former_folname="", folname=folnames[i], first=True, JsonDataSplit=JsonDataSplit, target_dataset=target_dataset, useStableDiffusion=useStableDiffusion, net=net, cons_reg=cons_reg)
        scores = inference(former_folname="", folname=folnames[i], JsonDataSplit=JsonDataSplit, target_dataset=target_dataset, useStableDiffusion=useStableDiffusion, net=net)
        write_to_csv(i+1, scores, csv_filename)
        save_mask(former_folname="", folname=folnames[i], JsonDataSplit=JsonDataSplit, target_dataset=target_dataset, useStableDiffusion=useStableDiffusion, net=net)
    else:
        train(former_folname=folnames[i-1], folname=folnames[i], first=False, alpha=alpha, JsonDataSplit=JsonDataSplit, target_dataset=target_dataset, useStableDiffusion=useStableDiffusion, net=net, cons_reg=cons_reg)
        scores = inference(former_folname=folnames[i-1], folname=folnames[i], JsonDataSplit=JsonDataSplit, target_dataset=target_dataset, useStableDiffusion=useStableDiffusion, net=net)
        write_to_csv(i+1, scores, csv_filename)
        save_mask(former_folname=folnames[i-1], folname=folnames[i], JsonDataSplit=JsonDataSplit, target_dataset=target_dataset, useStableDiffusion=useStableDiffusion, net=net)

In [ ]:
# Supervised Learning
project = "240208sup"
folname = project + '_iter1'
os.makedirs('results/'+ project +'_results', exist_ok=True)
csv_filename = 'results/'+project+'_results/results.csv'
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Iteration', 'F1-Score', 'Accuracy', 'Specificity', 'Recall', 'Precision'])

# params setting
target_dataset = 'chun'
net = 'deeplab'
epochs = 1000

train(former_folname="", folname=folname, first=True, target_dataset=target_dataset, net=net, epochs=epochs, supervised=True)
scores = inference(former_folname="", folname=folname, target_dataset=target_dataset, net=net, supervised=True)
write_to_csv(1, scores, csv_filename)